In [7]:
from functools import reduce
from typing import List

from nba_api.stats.endpoints import (
    LeagueDashPlayerStats,
    LeagueDashPlayerBioStats,
    TeamGameLogs,
)
import pandas as pd


In [2]:
df = pd.read_parquet("data/gamelog_2023.parquet")

In [4]:
df.columns

Index(['game_n', 'SEASON_YEAR', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME',
       'GAME_ID', 'GAME_DATE', 'MATCHUP', 'WL', 'MIN', 'FGM', 'FGA', 'FG_PCT',
       'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB',
       'REB', 'AST', 'TOV', 'STL', 'BLK', 'BLKA', 'PF', 'PFD', 'PTS',
       'PLUS_MINUS', 'GP_RANK', 'W_RANK', 'L_RANK', 'W_PCT_RANK', 'MIN_RANK',
       'FGM_RANK', 'FGA_RANK', 'FG_PCT_RANK', 'FG3M_RANK', 'FG3A_RANK',
       'FG3_PCT_RANK', 'FTM_RANK', 'FTA_RANK', 'FT_PCT_RANK', 'OREB_RANK',
       'DREB_RANK', 'REB_RANK', 'AST_RANK', 'TOV_RANK', 'STL_RANK', 'BLK_RANK',
       'BLKA_RANK', 'PF_RANK', 'PFD_RANK', 'PTS_RANK', 'PLUS_MINUS_RANK',
       'E_OFF_RATING', 'OFF_RATING', 'E_DEF_RATING', 'DEF_RATING',
       'E_NET_RATING', 'NET_RATING', 'AST_PCT', 'AST_TO', 'AST_RATIO',
       'OREB_PCT', 'DREB_PCT', 'REB_PCT', 'TM_TOV_PCT', 'EFG_PCT', 'TS_PCT',
       'E_PACE', 'PACE', 'PACE_PER40', 'POSS', 'PIE', 'OFF_RATING_RANK',
       'DEF_RATING_RANK', 'NET_RA

In [6]:
df[["TEAM_ABBREVIATION", "GAME_DATE", "MATCHUP", "OFF_RATING", "DEF_RATING"]]

,TEAM_ABBREVIATION,GAME_DATE,MATCHUP,OFF_RATING,DEF_RATING
0,LAC,2022-11-15T00:00:00,LAC @ DAL,103.0,115.3
1,NYK,2022-11-15T00:00:00,NYK @ UTA,140.4,111.0
2,SAS,2022-11-15T00:00:00,SAS @ POR,120.6,112.2
3,BKN,2022-11-15T00:00:00,BKN @ SAC,115.3,103.0
4,SAC,2022-11-15T00:00:00,SAC vs. BKN,107.8,114.6
...,...,...,...,...,...
415,NOP,2022-10-19T00:00:00,NOP @ BKN,109.1,106.1
416,PHI,2022-10-18T00:00:00,PHI @ BOS,97.3,107.0
417,BOS,2022-10-18T00:00:00,BOS vs. PHI,129.9,119.4
418,GSW,2022-10-18T00:00:00,GSW vs. LAL,107.0,97.3


In [51]:
def join(frames: List[pd.DataFrame]) -> pd.DataFrame:
    """
    join a list of dataframes, adding a _DROP suffix for repeated
    columns, which we can then filter out
    """
    return reduce(lambda x, y: x.join(y, rsuffix="_DROP"), frames).filter(
        regex="^(?!.*_DROP$)"
    )


In [10]:
season="2022-23"
logs = []
for measure in [None, "Advanced"]:
    logs.append(
        TeamGameLogs(
            season_nullable=season,
            measure_type_player_game_logs_nullable=measure,
        ).get_data_frames()[0]
    )

In [13]:
# things are still as we expect them here
logs[1][["TEAM_ABBREVIATION", "GAME_DATE", "MATCHUP", "OFF_RATING", "DEF_RATING"]]

,TEAM_ABBREVIATION,GAME_DATE,MATCHUP,OFF_RATING,DEF_RATING
0,MEM,2022-11-15T00:00:00,MEM @ NOP,103.0,115.3
1,SAC,2022-11-15T00:00:00,SAC vs. BKN,140.4,111.0
2,POR,2022-11-15T00:00:00,POR vs. SAS,120.6,112.2
3,NOP,2022-11-15T00:00:00,NOP vs. MEM,115.3,103.0
4,UTA,2022-11-15T00:00:00,UTA vs. NYK,107.8,114.6
...,...,...,...,...,...
415,TOR,2022-10-19T00:00:00,TOR vs. CLE,109.1,106.1
416,LAL,2022-10-18T00:00:00,LAL @ GSW,97.3,107.0
417,BOS,2022-10-18T00:00:00,BOS vs. PHI,129.9,119.4
418,GSW,2022-10-18T00:00:00,GSW vs. LAL,107.0,97.3


In [52]:
# already here they're effed, so the join function is what's wrong
games = join(logs)
games[["TEAM_ABBREVIATION", "GAME_DATE", "MATCHUP", "OFF_RATING", "DEF_RATING"]]

,TEAM_ABBREVIATION,GAME_DATE,MATCHUP,OFF_RATING,DEF_RATING
0,LAC,2022-11-15T00:00:00,LAC @ DAL,103.0,115.3
1,NYK,2022-11-15T00:00:00,NYK @ UTA,140.4,111.0
2,SAS,2022-11-15T00:00:00,SAS @ POR,120.6,112.2
3,BKN,2022-11-15T00:00:00,BKN @ SAC,115.3,103.0
4,SAC,2022-11-15T00:00:00,SAC vs. BKN,107.8,114.6
...,...,...,...,...,...
415,NOP,2022-10-19T00:00:00,NOP @ BKN,109.1,106.1
416,PHI,2022-10-18T00:00:00,PHI @ BOS,97.3,107.0
417,BOS,2022-10-18T00:00:00,BOS vs. PHI,129.9,119.4
418,GSW,2022-10-18T00:00:00,GSW vs. LAL,107.0,97.3


In [20]:
# this messes up our data! note that phi @ bos is no longer the same as bos @ phi, they should be opposites
logs[0].join(logs[1], rsuffix="_DROP")[["TEAM_ABBREVIATION", "GAME_DATE", "MATCHUP", "OFF_RATING", "DEF_RATING"]]

,TEAM_ABBREVIATION,GAME_DATE,MATCHUP,OFF_RATING,DEF_RATING
0,LAC,2022-11-15T00:00:00,LAC @ DAL,103.0,115.3
1,NYK,2022-11-15T00:00:00,NYK @ UTA,140.4,111.0
2,SAS,2022-11-15T00:00:00,SAS @ POR,120.6,112.2
3,BKN,2022-11-15T00:00:00,BKN @ SAC,115.3,103.0
4,SAC,2022-11-15T00:00:00,SAC vs. BKN,107.8,114.6
...,...,...,...,...,...
415,NOP,2022-10-19T00:00:00,NOP @ BKN,109.1,106.1
416,PHI,2022-10-18T00:00:00,PHI @ BOS,97.3,107.0
417,BOS,2022-10-18T00:00:00,BOS vs. PHI,129.9,119.4
418,GSW,2022-10-18T00:00:00,GSW vs. LAL,107.0,97.3


In [53]:
# I think the issue is that we're joining by index, but they're not ordered the same!
# let's try using "merge" instead of "join"
joined = pd.merge(logs[0], logs[1], how="inner", on=["TEAM_ID", "GAME_ID"], suffixes=("", "_DROP"))
joined = joined.filter(regex="^(?!.*_DROP$)")
joined

,SEASON_YEAR,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,...,AST_TO_RANK,AST_RATIO_RANK,OREB_PCT_RANK,DREB_PCT_RANK,REB_PCT_RANK,TM_TOV_PCT_RANK,EFG_PCT_RANK,TS_PCT_RANK,PACE_RANK,PIE_RANK
0,2022-23,1610612746,LAC,LA Clippers,0022200207,2022-11-15T00:00:00,LAC @ DAL,L,48.0,34,...,380,153,418,30,61,410,34,19,375,167
1,2022-23,1610612752,NYK,New York Knicks,0022200208,2022-11-15T00:00:00,NYK @ UTA,W,48.0,44,...,155,213,124,307,238,162,216,225,91,207
2,2022-23,1610612759,SAS,San Antonio Spurs,0022200209,2022-11-15T00:00:00,SAS @ POR,L,48.0,44,...,173,9,199,184,237,368,125,145,297,231
3,2022-23,1610612751,BKN,Brooklyn Nets,0022200210,2022-11-15T00:00:00,BKN @ SAC,L,48.0,40,...,281,201,417,332,413,256,140,69,13,394
4,2022-23,1610612758,SAC,Sacramento Kings,0022200210,2022-11-15T00:00:00,SAC vs. BKN,W,48.0,56,...,77,5,89,4,8,256,4,3,13,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,2022-23,1610612740,NOP,New Orleans Pelicans,0022200006,2022-10-19T00:00:00,NOP @ BKN,W,48.0,50,...,59,109,12,105,10,126,171,202,131,32
416,2022-23,1610612755,PHI,Philadelphia 76ers,0022200001,2022-10-18T00:00:00,PHI @ BOS,L,48.0,40,...,370,397,377,159,354,195,98,68,297,340
417,2022-23,1610612738,BOS,Boston Celtics,0022200001,2022-10-18T00:00:00,BOS vs. PHI,W,48.0,46,...,87,170,262,43,67,74,25,21,297,81
418,2022-23,1610612744,GSW,Golden State Warriors,0022200002,2022-10-18T00:00:00,GSW vs. LAL,W,48.0,45,...,194,142,221,141,157,259,209,236,1,117


In [55]:
# hey now things seem to work!
joined[["TEAM_ABBREVIATION", "GAME_DATE", "MATCHUP", "OFF_RATING", "DEF_RATING"]]

,TEAM_ABBREVIATION,GAME_DATE,MATCHUP,OFF_RATING,DEF_RATING
0,LAC,2022-11-15T00:00:00,LAC @ DAL,106.3,109.6
1,NYK,2022-11-15T00:00:00,NYK @ UTA,114.6,107.8
2,SAS,2022-11-15T00:00:00,SAS @ POR,112.2,120.6
3,BKN,2022-11-15T00:00:00,BKN @ SAC,111.0,140.4
4,SAC,2022-11-15T00:00:00,SAC vs. BKN,140.4,111.0
...,...,...,...,...,...
415,NOP,2022-10-19T00:00:00,NOP @ BKN,127.5,105.9
416,PHI,2022-10-18T00:00:00,PHI @ BOS,119.4,129.9
417,BOS,2022-10-18T00:00:00,BOS vs. PHI,129.9,119.4
418,GSW,2022-10-18T00:00:00,GSW vs. LAL,107.0,97.3


In [56]:
def join2(frames: List[pd.DataFrame], on: List[str]) -> pd.DataFrame:
    """
    join a list of dataframes, adding a _DROP suffix for repeated
    columns, which we can then filter out
    """
    return reduce(lambda x, y: x.merge(y, on=on, suffixes=("", "_DROP")), frames).filter(
        regex="^(?!.*_DROP$)"
    )


In [57]:
df = join2([logs[0], logs[1]], on=["TEAM_ID", "GAME_ID"])
df

,SEASON_YEAR,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,...,AST_TO_RANK,AST_RATIO_RANK,OREB_PCT_RANK,DREB_PCT_RANK,REB_PCT_RANK,TM_TOV_PCT_RANK,EFG_PCT_RANK,TS_PCT_RANK,PACE_RANK,PIE_RANK
0,2022-23,1610612746,LAC,LA Clippers,0022200207,2022-11-15T00:00:00,LAC @ DAL,L,48.0,34,...,380,153,418,30,61,410,34,19,375,167
1,2022-23,1610612752,NYK,New York Knicks,0022200208,2022-11-15T00:00:00,NYK @ UTA,W,48.0,44,...,155,213,124,307,238,162,216,225,91,207
2,2022-23,1610612759,SAS,San Antonio Spurs,0022200209,2022-11-15T00:00:00,SAS @ POR,L,48.0,44,...,173,9,199,184,237,368,125,145,297,231
3,2022-23,1610612751,BKN,Brooklyn Nets,0022200210,2022-11-15T00:00:00,BKN @ SAC,L,48.0,40,...,281,201,417,332,413,256,140,69,13,394
4,2022-23,1610612758,SAC,Sacramento Kings,0022200210,2022-11-15T00:00:00,SAC vs. BKN,W,48.0,56,...,77,5,89,4,8,256,4,3,13,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,2022-23,1610612740,NOP,New Orleans Pelicans,0022200006,2022-10-19T00:00:00,NOP @ BKN,W,48.0,50,...,59,109,12,105,10,126,171,202,131,32
416,2022-23,1610612755,PHI,Philadelphia 76ers,0022200001,2022-10-18T00:00:00,PHI @ BOS,L,48.0,40,...,370,397,377,159,354,195,98,68,297,340
417,2022-23,1610612738,BOS,Boston Celtics,0022200001,2022-10-18T00:00:00,BOS vs. PHI,W,48.0,46,...,87,170,262,43,67,74,25,21,297,81
418,2022-23,1610612744,GSW,Golden State Warriors,0022200002,2022-10-18T00:00:00,GSW vs. LAL,W,48.0,45,...,194,142,221,141,157,259,209,236,1,117


In [58]:
df[["TEAM_ABBREVIATION", "GAME_DATE", "MATCHUP", "OFF_RATING", "DEF_RATING"]]

,TEAM_ABBREVIATION,GAME_DATE,MATCHUP,OFF_RATING,DEF_RATING
0,LAC,2022-11-15T00:00:00,LAC @ DAL,106.3,109.6
1,NYK,2022-11-15T00:00:00,NYK @ UTA,114.6,107.8
2,SAS,2022-11-15T00:00:00,SAS @ POR,112.2,120.6
3,BKN,2022-11-15T00:00:00,BKN @ SAC,111.0,140.4
4,SAC,2022-11-15T00:00:00,SAC vs. BKN,140.4,111.0
...,...,...,...,...,...
415,NOP,2022-10-19T00:00:00,NOP @ BKN,127.5,105.9
416,PHI,2022-10-18T00:00:00,PHI @ BOS,119.4,129.9
417,BOS,2022-10-18T00:00:00,BOS vs. PHI,129.9,119.4
418,GSW,2022-10-18T00:00:00,GSW vs. LAL,107.0,97.3


In [62]:
players = pd.read_parquet("data/players_2023.parquet")
players

,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,...,FGM_PG_RANK,FGA_PG_RANK,PLAYER_HEIGHT,PLAYER_HEIGHT_INCHES,PLAYER_WEIGHT,COLLEGE,COUNTRY,DRAFT_YEAR,DRAFT_ROUND,DRAFT_NUMBER
0,1631260,AJ Green,AJ,1610612749,MIL,23.0,1,1,0,1.000,...,430,447,6-5,77,190,Northern Iowa,USA,Undrafted,Undrafted,Undrafted
1,1631100,AJ Griffin,AJ,1610612737,ATL,19.0,10,5,5,0.500,...,227,243,6-6,78,220,Duke,USA,2022,1,16
2,203932,Aaron Gordon,Aaron,1610612743,DEN,27.0,13,9,4,0.692,...,90,122,6-8,80,235,Arizona,USA,2014,1,4
3,1628988,Aaron Holiday,Aaron,1610612737,ATL,26.0,14,9,5,0.643,...,281,274,6-0,72,185,UCLA,USA,2018,1,23
4,1630174,Aaron Nesmith,Aaron,1610612754,IND,23.0,8,3,5,0.375,...,222,193,6-5,77,215,Vanderbilt,USA,2020,1,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451,1629139,Yuta Watanabe,Yuta,1610612751,BKN,28.0,12,3,9,0.250,...,233,271,6-8,80,215,George Washington,Japan,Undrafted,Undrafted,Undrafted
452,1628380,Zach Collins,Zach,1610612759,SAS,24.0,9,5,4,0.556,...,172,198,6-11,83,250,Gonzaga,USA,2017,1,10
453,203897,Zach LaVine,Zach,1610612741,CHI,27.0,10,5,5,0.500,...,36,32,6-5,77,200,UCLA,USA,2014,1,13
454,1630192,Zeke Nnaji,Zeke,1610612743,DEN,21.0,8,5,3,0.625,...,382,390,6-9,81,240,Arizona,USA,2020,1,22
